# Tables demo/ workbook 



In [48]:
# INITIALIZATION BOILERPLATE

# The Jupyter kernel for this notebook usually starts up inside the notebooks
# directory, but the text_extensions_for_pandas package code is in the parent
# directory. Add that parent directory to the front of the Python include path.
import sys
if (sys.path[0] != ".."):
    sys.path[0] = ".."

import json
import os
from ibm_watson import CompareComplyV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException

import pandas as pd 
import regex
import text_extensions_for_pandas as tp
from IPython.core.display import HTML



In [2]:
def init_watson_table_api():
    # Retrieve the APIKEY for authentication
    apikey = os.environ.get("IBM_API_KEY_TABLES")
    if apikey is None:
        raise ValueError("Expected apikey in the environment variable 'IBM_API_KEY'")

    # Get the service URL for your IBM Cloud instance
    ibm_cloud_service_url = os.environ.get("IBM_SERVICE_URL_TABLES")
    if ibm_cloud_service_url is None:
        raise ValueError("Expected IBM cloud service URL in the environment variable 'IBM_SERVICE_URL'")

        #initialize the authenticator
    authenticator = IAMAuthenticator(apikey)
    compare_comply = CompareComplyV1(
        version = '2020-1-1',
        authenticator=authenticator
    )

    compare_comply.set_service_url(ibm_cloud_service_url)
    return compare_comply

In [3]:
base_example_path = "../resources/tables/"
file_names = [ "who_covid_report_table", "20-populous-countries", "cali-temp-chart", "california-population-chart", "double_header_table"]

archive_name = "archive"
#set to true to query watson
re_query_watson = False



In [4]:
#responses_dict = {}
if re_query_watson:
    print("This shouldn't happen")
    compare_comply = init_watson_table_api()
    for file_n in file_names:
        with open(f"{base_example_path}{file_n}.pdf", 'rb') as base_file:
            try:
                result = compare_comply.extract_tables(base_file).get_result()
                responses_dict[file_n] = result
            except ApiException as ex:
                print ("Method failed with status code " + str(ex.code) + ": " + ex.message)
     #archive file
    with open(f"{base_example_path}{archive_name}.json", 'w') as archive_file:
        json.dump(responses_dict, archive_file)
else:
    with open(f"{base_example_path}{archive_name}.json", 'r') as archive_file:
        responses_dict = json.load(archive_file)

In [5]:
#select a response and process it, printing the outputs
response = responses_dict["double_header_table"]

#most basic work flow for processing tables
dfs_dict = tp.watson_tables_parse_response(response)
table = tp.make_table(dfs_dict)
table

Nine months ended setptember 30         \
                                                2004   2005   
Dividends received                              4.7%  15.4%   
IRS audit settlement                           15.2%    58%   
Statatory tax rate                               38%    37%   
Total tax rate                                 15.1%  38.8%   

                     Three months ended setptember 30         
                                                 2004   2005  
Dividends received                               3.3%  13.2%  
IRS audit settlement                            35.5%    97%  
Statatory tax rate                                36%    35%  
Total tax rate                                   4.3%  76.1%

# Break down process of reconstructing table: 

The raw JSON output is omitted for brevity

The first stage we see is the pandas-ified deconstructed table. This gives a good idea of the schema of data we are dealing with. It can be reached by running the ``` tp.watson_tables_parse_response(...)``` command. 
It contains information about the row headers, column headers and body cells and their contents


Next we have the value-attribute correlated table. This phase correlates each value wits respective row and header cells using information about cell ID' row and column numbers and other information from the original table. 

In [6]:
#select a response and process it, printing the outputs
response = responses_dict["double_header_table"]



print("displaying row headers:")
display(dfs_dict['row_headers'])
print("displaying column headers:")
display(dfs_dict['col_headers'])
print("displaying objects:")
display(dfs_dict['body_cells'].head(30))

displaying row headers:


,text,column_index_begin,column_index_end,row_index_begin,row_index_end,cell_id,text_normalized
0,Statatory tax rate,0,0,2,2,rowHeader-2810-2829,Statatory tax rate
1,IRS audit settlement,0,0,3,3,rowHeader-4068-4089,IRS audit settlement
2,Dividends received,0,0,4,4,rowHeader-5329-5348,Dividends received
3,Total tax rate,0,0,5,5,rowHeader-6586-6601,Total tax rate


displaying column headers:


,text,column_index_begin,column_index_end,row_index_begin,row_index_end,cell_id,text_normalized
0,,0,0,0,0,colHeader-786-787,
1,Three months ended setptember 30,1,2,0,0,colHeader-1012-1206,Three months ended setptember 30
2,Nine months ended setptember 30,3,4,0,0,colHeader-1444-1514,Nine months ended setptember 30
3,,0,0,1,1,colHeader-1586-1587,
4,2005,1,1,1,1,colHeader-1813-1818,2005
5,2004,2,2,1,1,colHeader-2061-2066,2004
6,2005,3,3,1,1,colHeader-2305-2310,2005
7,2004,4,4,1,1,colHeader-2553-2558,2004


displaying objects:


,text,column_index_begin,column_index_end,row_index_begin,row_index_end,cell_id,column_header_ids,column_header_texts,row_header_ids,row_header_texts,attributes.text,attributes.type
0,35%,1,1,2,2,bodyCell-3073-3077,"[colHeader-1012-1206, colHeader-1813-1818]","[Three months ended setptember 30, 2005]",[rowHeader-2810-2829],[Statatory tax rate],[35%],[Percentage]
1,36%,2,2,2,2,bodyCell-3320-3324,"[colHeader-1012-1206, colHeader-2061-2066]","[Three months ended setptember 30, 2004]",[rowHeader-2810-2829],[Statatory tax rate],[36%],[Percentage]
2,37%,3,3,2,2,bodyCell-3564-3568,"[colHeader-1444-1514, colHeader-2305-2310]","[Nine months ended setptember 30, 2005]",[rowHeader-2810-2829],[Statatory tax rate],[37%],[Percentage]
3,38%,4,4,2,2,bodyCell-3811-3815,"[colHeader-1444-1514, colHeader-2553-2558]","[Nine months ended setptember 30, 2004]",[rowHeader-2810-2829],[Statatory tax rate],[38%],[Percentage]
4,97%,1,1,3,3,bodyCell-4333-4337,"[colHeader-1012-1206, colHeader-1813-1818]","[Three months ended setptember 30, 2005]",[rowHeader-4068-4089],[IRS audit settlement],[97%],[Percentage]
5,35.5%,2,2,3,3,bodyCell-4579-4585,"[colHeader-1012-1206, colHeader-2061-2066]","[Three months ended setptember 30, 2004]",[rowHeader-4068-4089],[IRS audit settlement],[35.5%],[Percentage]
6,58%,3,3,3,3,bodyCell-4825-4829,"[colHeader-1444-1514, colHeader-2305-2310]","[Nine months ended setptember 30, 2005]",[rowHeader-4068-4089],[IRS audit settlement],[58%],[Percentage]
7,15.2%,4,4,3,3,bodyCell-5071-5077,"[colHeader-1444-1514, colHeader-2553-2558]","[Nine months ended setptember 30, 2004]",[rowHeader-4068-4089],[IRS audit settlement],[15.2%],[Percentage]
8,13.2%,1,1,4,4,bodyCell-5591-5597,"[colHeader-1012-1206, colHeader-1813-1818]","[Three months ended setptember 30, 2005]",[rowHeader-5329-5348],[Dividends received],[13.2%],[Percentage]
9,3.3%,2,2,4,4,bodyCell-5838-5843,"[colHeader-1012-1206, colHeader-2061-2066]","[Three months ended setptember 30, 2004]",[rowHeader-5329-5348],[Dividends received],[3.3%],[Percentage]


In [7]:
table_exploded, row_headers, col_headers = tp.make_exploded_df(dfs_dict)
display(table_exploded)

,text,row_header_texts_0,column_header_texts_0,column_header_texts_1
0,35%,Statatory tax rate,Three months ended setptember 30,2005
1,36%,Statatory tax rate,Three months ended setptember 30,2004
2,37%,Statatory tax rate,Nine months ended setptember 30,2005
3,38%,Statatory tax rate,Nine months ended setptember 30,2004
4,97%,IRS audit settlement,Three months ended setptember 30,2005
5,35.5%,IRS audit settlement,Three months ended setptember 30,2004
6,58%,IRS audit settlement,Nine months ended setptember 30,2005
7,15.2%,IRS audit settlement,Nine months ended setptember 30,2004
8,13.2%,Dividends received,Three months ended setptember 30,2005
9,3.3%,Dividends received,Three months ended setptember 30,2004


## Compare the reconstructed table with the extracted html from table understanding output

We run the final step of table reconstruction: `tp.make_table`. 
Note we could have run `make_table_from_exploded_dfs` here instead with the output of the last step. 

Additionally we render the html that is produced in an intermediate step by the table extraction api to give a comparison of the original vs reconstructed table

In [8]:
dfs_dict = tp.watson_tables_parse_response(response)
table = tp.make_table(dfs_dict)
display(table)
display(HTML(response["document"]["html"]))

Nine months ended setptember 30         \
                                                2004   2005   
Dividends received                              4.7%  15.4%   
IRS audit settlement                           15.2%    58%   
Statatory tax rate                               38%    37%   
Total tax rate                                 15.1%  38.8%   

                     Three months ended setptember 30         
                                                 2004   2005  
Dividends received                               3.3%  13.2%  
IRS audit settlement                            35.5%    97%  
Statatory tax rate                                36%    35%  
Total tax rate                                   4.3%  76.1%

### Data manipulation while in pandas dataframe: 

Once the table is processed through into a pandas dataframe, it can be easily manipulated like any other pandas dataframe. 

Here, we show selecting rows, columns, specific heading combinations, and even specific items. Each of these is an easy to use one-liner

In [9]:
#change names of heading feilds
table= table.rename_axis(columns=["period","year"])
#display whole table
display(table)

period               Nine months ended setptember 30         \
year                                            2004   2005   
Dividends received                              4.7%  15.4%   
IRS audit settlement                           15.2%    58%   
Statatory tax rate                               38%    37%   
Total tax rate                                 15.1%  38.8%   

period               Three months ended setptember 30         
year                                             2004   2005  
Dividends received                               3.3%  13.2%  
IRS audit settlement                            35.5%    97%  
Statatory tax rate                                36%    35%  
Total tax rate                                   4.3%  76.1%

In [10]:
#select just one set duration
display(table.loc[:,"Nine months ended setptember 30"])

year,2004,2005
Dividends received,4.7%,15.4%
IRS audit settlement,15.2%,58%
Statatory tax rate,38%,37%
Total tax rate,15.1%,38.8%


In [11]:
# One specific time period 
display(table[("Nine months ended setptember 30","2005")].to_frame())

,Nine months ended setptember 30
,2005
Dividends received,15.4%
IRS audit settlement,58%
Statatory tax rate,37%
Total tax rate,38.8%


In [12]:
# Display data from just one year
display(table.swaplevel(axis=1)["2004"])

period,Nine months ended setptember 30,Three months ended setptember 30
Dividends received,4.7%,3.3%
IRS audit settlement,15.2%,35.5%
Statatory tax rate,38%,36%
Total tax rate,15.1%,4.3%


In [13]:
#select one specific field
display(table.loc["Dividends received",:].to_frame())

Dividends received
period                           year                   
Nine months ended setptember 30  2004               4.7%
                                 2005              15.4%
Three months ended setptember 30 2004               3.3%
                                 2005              13.2%

### By converting some elements to numeric datatypes additional analysis is possible

Currently, one additional step is required to convert the datatypes of numeric elements from strings or percents to numbers; (currently working on automating it) 

Doing this, however enables rich analysis of the data provided, by leveraging the powerful data analytics side of pandas.
Here we show taking generating general descriptive statistics for the set; using `DataFrame.mean()`, `DataFrame.std()` and `DataFrame.describe()` however more in depth analytics are possible

In [14]:
#convert strings to ints. 
temp = table.copy()

temp = tp.convert_cols_to_numeric(temp)
display(temp)

period               Nine months ended setptember 30        \
year                                            2004  2005   
Dividends received                               4.7  15.4   
IRS audit settlement                            15.2  58.0   
Statatory tax rate                              38.0  37.0   
Total tax rate                                  15.1  38.8   

period               Three months ended setptember 30        
year                                             2004  2005  
Dividends received                                3.3  13.2  
IRS audit settlement                             35.5  97.0  
Statatory tax rate                               36.0  35.0  
Total tax rate                                    4.3  76.1

In [15]:
#now other analyses are possible
print("\nShow means by year")
display(temp.mean(axis=0).to_frame())


Show means by year


0
period                           year        
Nine months ended setptember 30  2004  18.250
                                 2005  37.300
Three months ended setptember 30 2004  19.775
                                 2005  55.325

In [16]:
print("\n\nShow means by subtype")
display(temp.mean(axis=1).to_frame())



Show means by subtype


,0
Dividends received,9.150
IRS audit settlement,51.425
Statatory tax rate,36.500
Total tax rate,33.575


In [17]:
print("\n\nShow the deviation for each category")
display(temp.std(axis=1).to_frame())



Show the deviation for each category


,0
Dividends received,6.041247
IRS audit settlement,35.053138
Statatory tax rate,1.290994
Total tax rate,31.801612


In [18]:
print("\n\nGenerate general descriptive statistics for the whole dataFrame")
print("By time period:")
display(temp.describe(percentiles =[]))
print("By category:")
temp.transpose().describe(percentiles=[])



Generate general descriptive statistics for the whole dataFrame
By time period:


period Nine months ended setptember 30             \
year                              2004       2005   
count                         4.000000   4.000000   
mean                         18.250000  37.300000   
std                          14.058094  17.420677   
min                           4.700000  15.400000   
50%                          15.150000  37.900000   
max                          38.000000  58.000000   

period Three months ended setptember 30             
year                               2004       2005  
count                          4.000000   4.000000  
mean                          19.775000  55.325000  
std                           18.451987  38.105238  
min                            3.300000  13.200000  
50%                           19.900000  55.550000  
max                           36.000000  97.000000

By category:


,Dividends received,IRS audit settlement,Statatory tax rate,Total tax rate
count,4.000000,4.000000,4.000000,4.000000
mean,9.150000,51.425000,36.500000,33.575000
std,6.041247,35.053138,1.290994,31.801612
min,3.300000,15.200000,35.000000,4.300000
50%,8.950000,46.750000,36.500000,26.950000
max,15.400000,97.000000,38.000000,76.100000


## Application example: extracting data from WHO Covid report

This is an example of how you can use the the package in an end to end use to rapidly charactarize a set of data from pdf format

In [19]:
response_covid = responses_dict["who_covid_report_table"]
num_tabs = len(response_covid.get("tables", []))

df = pd.DataFrame()

for i in range (num_tabs):
    dfs_dict = tp.watson_tables_parse_response(response_covid, table_number=i)
    next_table = tp.make_table(dfs_dict,concat_with='', col_explode_by = "concat")
    #display(next_table)
    df = df.append(next_table, ignore_index = True)

In [20]:
#use pandas functionality to remove bad data (~5 rows that were misinterpereted by table extraction as data when they were sub-headers)
rows_to_keep = [r.to_list().count('') == 0 for _,r in df.iterrows()] 
df = df[rows_to_keep]

In [21]:
df_converted = tp.convert_cols_to_numeric(df, columns=["Days since last reported case","Total confirmed cases", "Total confirmed new cases", 'Total new deaths '], conv_to= 'int')


ERROR READING VALUE:"-"	 Filling with <NA>
ERROR READING VALUE:"-"	 Filling with <NA>


In [22]:
df_converted

,Days since last reported case,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification i
1,0,South Africa,287796,11554,4 172,93,Community transmission
2,0,Nigeria,33153,595,744,4,Community transmission
3,0,Ghana,24988,470,139,0,Community transmission
4,0,Algeria,19689,494,1 018,7,Community transmission
5,1,Cameroon,15173,0,359,0,Community transmission
...,...,...,...,...,...,...,...
222,2,Guam,304,0,5,0,Clusters of cases
223,16,French Polynesia,62,0,0,0,Sporadic cases
225,2,(Commonwealth of the),33,0,2,0,Pending
226,34,New Caledonia,21,0,0,0,Sporadic cases


In [23]:
df_converted.describe()

,Total confirmed cases,Total confirmed new cases,Total new deaths
count,2.160000e+02,216.000000,216.000000
mean,6.002226e+04,910.995370,16.824074
std,2.722975e+05,4904.276433,68.539831
min,2.000000e+00,0.000000,0.000000
25%,3.342500e+02,0.000000,0.000000
50%,2.429000e+03,24.500000,0.000000
75%,2.467450e+04,291.500000,3.250000
max,3.286063e+06,60113.000000,631.000000
